In [1]:
cd ..

/Users/minghuang/Documents/git/SEA_traffic_accident_prediction


In [23]:
from data_processing_scripts import socrata_api_wrapper as saw
from data_processing_scripts import psql_helper as ph 
from data_processing_scripts import footballdb_scraper as fs
from data_processing_scripts import baseball_almanac_scraper as bas
from data_processing_scripts import wunderground_scraper as ws
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import psycopg2
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.cross_validation import cross_val_score, train_test_split
%matplotlib inline

In [3]:
query = 'select * from compiled_data3;'

In [4]:
conn = psycopg2.connect(dbname='traffic_accident', user='minghuang', host='localhost')
df = pd.read_sql_query(query, con=conn)
conn.close()

In [5]:
df.sort(axis=1).head().T

,0,1,2,3,4
condition_id,2.00,2.00,2.00,2.00,2.00
dewpoint,32.00,35.10,37.00,37.90,39.90
dow,0.00,0.00,0.00,0.00,0.00
have_gustspeed,0.00,0.00,0.00,0.00,0.00
have_heatindex,0.00,0.00,0.00,0.00,0.00
have_precip,0.00,0.00,0.00,0.00,0.00
hour,0.00,0.00,0.00,0.00,0.00
humidity,82.00,74.00,74.00,76.00,59.00
intersections,302.00,302.00,302.00,302.00,302.00
mariner_plays,0.00,0.00,0.00,0.00,0.00


In [6]:
df.count()

zone_beat_id        1202376
condition_id        1202376
winddir_id          1202376
intersections       1202376
hour                1202376
dow                 1202376
month               1202376
mariner_plays       1202376
seahawk_plays       1202376
dewpoint            1202376
have_gustspeed      1202376
have_heatindex      1202376
humidity            1202376
have_precip         1202376
pressure            1202376
temp                1202376
visibility          1202376
windspeed           1202376
number_accidents    1202376
dtype: int64

In [7]:
print 'Mean:', df.number_accidents.mean()
print 'Var:', df.number_accidents.var()

Mean: 0.0332325329182
Var: 0.0399693061222


In [8]:
df2 = df.sample(100000)

In [9]:
y = df2.pop('number_accidents').values
X = df2.sort(axis=1).values

In [12]:
encoder = OneHotEncoder(categorical_features=[0, 2, 6, 10, 15, 17])
mat = encoder.fit_transform(X)

In [13]:
scaler = StandardScaler(with_std=False)
mat2 = scaler.fit_transform(mat.toarray())

In [14]:
X_const = sm.add_constant(mat2, prepend=False)

In [15]:
nbinom_model = sm.GLM(y, X_const, family=sm.families.NegativeBinomial())

In [16]:
res = nbinom_model.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99859
Model Family:        NegativeBinomial   Df Model:                          140
Link Function:                    log   Scale:                   1.10835900006
Method:                          IRLS   Log-Likelihood:                -13738.
Date:                Tue, 29 Sep 2015   Deviance:                       19067.
Time:                        13:20:04   Pearson chi2:                 1.11e+05
No. Iterations:                    23                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.0240   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x2           -17.1760   1.88e+05  -9.16e-05      1.0

In [17]:
poisson_model = sm.GLM(y, X_const, family=sm.families.Poisson())

In [18]:
res_pois = poisson_model.fit()
print(res_pois.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99859
Model Family:                 Poisson   Df Model:                          140
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -13910.
Date:                Tue, 29 Sep 2015   Deviance:                       21793.
Time:                        13:20:56   Pearson chi2:                 1.14e+05
No. Iterations:                    10                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.0594     24.218      0.126      0.899     -44.407      50.526
x2            -8.1154    265.473     -0.031      0.9

In [58]:
encoder.feature_indices_

array([  0,  29,  36,  60,  73,  92, 144])

In [59]:
encoder.categorical_features

[0, 2, 6, 10, 15, 17]

In [57]:
place = -1
for i, col in enumerate(df2.sort(axis=1).columns):
    place += 1
    if i in encoder.categorical_features:
        num = df[col].max() - df[col].min() + 1
        print num, 'features', place, place + num - 1
        place += num - 1
    else:
        print 'continuous', place

28 features 0 27
continuous 28
7.0 features 29 35.0
continuous 36.0
continuous 37.0
continuous 38.0
24.0 features 39.0 62.0
continuous 63.0
continuous 64.0
continuous 65.0
12.0 features 66.0 77.0
continuous 78.0
continuous 79.0
continuous 80.0
continuous 81.0
18 features 82.0 99.0
continuous 100.0
51 features 101.0 151.0


In [60]:
res.aic

36675.020696991262

In [19]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99859
Model Family:        NegativeBinomial   Df Model:                          140
Link Function:                    log   Scale:                   1.10835900006
Method:                          IRLS   Log-Likelihood:                -13738.
Date:                Tue, 29 Sep 2015   Deviance:                       19067.
Time:                        13:21:08   Pearson chi2:                 1.11e+05
No. Iterations:                    23                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.0240   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x2           -17.1760   1.88e+05  -9.16e-05      1.000   -3.68e+05    3.68e+05
x3             7.1996   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x4             7.1777   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x5             7.0337   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x6           -16.6270   1.12e+05     -0.000      1.000   -2.21e+05     2.2e+05
x7             8.0838   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x8           -17.1927   1.98e+05  -8.69e-05      1.000   -3.88e+05    3.88e+05
x9             6.9897   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x10          -16.3263   1.64e+05  -9.95e-05      1.000   -3.21e+05    3.21e+05
x11          -16.5361   4.89e+04     -0.000      1.000   -9.59e+04    9.58e+04
x12          -16.6423   1.06e+05     -0.000      1.000   -2.08e+05    2.08e+05
x13            6.7787   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x14            7.2018   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x15            7.2081   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x16            7.2927   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x17            7.5686   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x18            6.7595   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x19            7.2116   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x20            6.7118   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x21            7.8593   1.71e+04      0.000      1.000   -3.36e+04    3.36e+04
x22            8.5889   1.71e+04      0.001      1.000   -3.36e+04    3.36e+04
x23          -16.1893   1.27e+05     -0.000      1.000   -2.49e+05    2.49e+05
x24           -0.0772      0.050     -1.544      0.123      -0.175       0.021
x25           -0.1158      0.050     -2.339      0.019      -0.213      -0.019
x26           -0.0010      0.047     -0.022      0.982      -0.093       0.091
x27           -0.0111      0.047     -0.235      0.814      -0.103       0.081
x28            0.0656      0.046      1.434      0.152      -0.024       0.155
x29            0.1775      0.043      4.085      0.000       0.092       0.263
x30           -0.0380      0.048     -0.797      0.425      -0.131       0.055
x31            0.2001      0.089      2.247      0.025       0.026       0.375
x32            0.1703      0.089      1.905      0.057      -0.005       0.345
x33           -0.1268      0.103     -1.230      0.219      -0.329       0.075
x34           -0.5229      0.121     -4.308      0.000      -0.761      -0.285
x35           -0.5787      0.124     -4.684      0.000      -0.821      -0.337
x36           -0.6547      0.131     -4.991      0.000      -0.912      -0.398
x37           -0.5393      0.124     -4.333      0.000      -0.783      -0.295
x3

In [20]:
res.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Generalized linear model
==================================================================
Model:              GLM              AIC:            27758.8431   
Link Function:      log              BIC:            -1130601.8370
Dependent Variable: y                Log-Likelihood: -13738.      
Date:               2015-09-29 13:21 LL-Null:        -14366.      
No. Observations:   100000           Deviance:       19067.       
Df Model:           140              Pearson chi2:   1.11e+05     
Df Residuals:       99859            Scale:          1.1084       
Method:             IRLS                                          
------------------------------------------------------------------
       Coef.     Std.Err.     z    P>|z|     [0.025       0.975]  
------------------------------------------------------------------
x1      7.0240  17123.4057  0.0004 0.9997  -33554.2345  33568.2826
x2    -17.1760 187530.6139 -0.0001 0.9999 -367570.4254 367536.0733
x3      7.1996  17123.4057  0.0004 0.9997  -33554.0589  33568.4582
x4      7.1777  17123.4057  0.0004 0.9997  -33554.0809  33568.4362
x5      7.0337  17123.4058  0.0004 0.9997  -33554.2249  33568.2923
x6    -16.6270 112498.6160 -0.0001 0.9999 -220509.8625 220476.6086
x7      8.0838  17123.4058  0.0005 0.9996  -33553.1748  33569.3424
x8    -17.1927 197942.5448 -0.0001 0.9999 -387977.4516 387943.0662
x9      6.9897  17123.4057  0.0004 0.9997  -33554.2688  33568.2482
x10   -16.3263 164021.1769 -0.0001 0.9999 -321491.9258 321459.2732
x11   -16.5361  48896.2154 -0.0003 0.9997  -95851.3573  95818.2852
x12   -16.6423 105994.6513 -0.0002 0.9999 -207762.3413 207729.0568
x13     6.7787  17123.4057  0.0004 0.9997  -33554.4799  33568.0372
x14     7.2018  17123.4057  0.0004 0.9997  -33554.0567  33568.4604
x15     7.2081  17123.4057  0.0004 0.9997  -33554.0504  33568.4666
x16     7.2927  17123.4057  0.0004 0.9997  -33553.9659  33568.5512
x17     7.5686  17123.4058  0.0004 0.9996  -33553.6900  33568.8272
x18     6.7595  17123.4057  0.0004 0.9997  -33554.4990  33568.0181
x19     7.2116  17123.4057  0.0004 0.9997  -33554.0469  33568.4701
x20     6.7118  17123.4058  0.0004 0.9997  -33554.5468  33567.9704
x21     7.8593  17123.4057  0.0005 0.9996  -33553.3993  33569.1178
x22     8.5889  17123.4057  0.0005 0.9996  -33552.6696  33569.8475
x23   -16.1893 127042.6492 -0.0001 0.9999 -249015.2062 248982.8277
x24    -0.0772      0.0500 -1.5443 0.1225      -0.1752      0.0208
x25    -0.1158      0.0495 -2.3394 0.0193      -0.2128     -0.0188
x26    -0.0010      0.0470 -0.0220 0.9825      -0.0932      0.0912
x27    -0.0111      0.0471 -0.2349 0.8143      -0.1033      0.0812
x28     0.0656      0.0457  1.4335 0.1517      -0.0241      0.1552
x29     0.1775      0.0435  4.0848 0.0000       0.0923      0.2626
x30    -0.0380      0.0476 -0.7974 0.4252      -0.1312      0.0553
x31     0.2001      0.0890  2.2468 0.0247       0.0255      0.3746
x32     0.1703      0.0894  1.9049 0.0568      -0.0049      0.3454
x33    -0.1268      0.1031 -1.2302 0.2186      -0.3289      0.0752
x34    -0.5229      0.1214 -4.3083 0.0000      -0.7608     -0.2850
x35    -0.5787      0.1235 -4.6838 0.0000      -0.8208     -0.3365
x36    -0.6547      0.1312 -4.9912 0.0000      -0.9118     -0.3976
x37    -0.5393      0.1245 -4.3327 0.0000      -0.7832     -0.2953
x38    -0.2576      0.1069 -2.4085 0.0160      -0.4672     -0.0480
x39     0.1162      0.0898  1.2934 0.1959      -0.0599      0.2922
x40    -0.1566      0.1011 -1.5486 0.1215      -0.3548      0.0416
x41    -0.1368      0.1005 -1.3623 0.1731      -0.3337      0.0600
x42    -0.2233      0.1046 -2.1344 0.0328      -0.4283     -0.0182
x43    -0.3075      0.1088 -2.8267 0.0047      -0.5207     -0.0943
x44     0.2983      0.0848  3.5189 0.0004       0.1321      0.4644
x45     0.1019      0.0923  1.1045 0.2694      -0.0789      0.2827
x46     0.1384      0.0894  1.5482 0.1216      -0.0368      0.3137
x47     0.1398

In [22]:
linlasso = Lasso()

In [25]:
cvs = cross_val_score(linlasso, mat2, y, scoring='r2', cv=5)

In [26]:
cvs

array([ -3.89830532e-05,  -2.81738059e-06,  -1.13373911e-04,
        -3.89142901e-04,  -2.38582518e-06])